<a href="https://colab.research.google.com/github/lucia1299/DBMS_InvestigativeJournalism/blob/main/MergingTriples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Merge the triple files from the two processing pipelines

In [ ]:
import json

no_label_json = '/content/drive/MyDrive/Colab Notebooks/TESI/github/example_API_triples_nolabel.json'
label_json = '/content/drive/MyDrive/Colab Notebooks/TESI/github/example_triples_withlabel.json'

merged_json = '/content/drive/MyDrive/Colab Notebooks/TESI/github/merged_example_triples.json'

with open(no_label_json, 'r') as f1, open(label_json, 'r') as f2:
    no_label_data = json.load(f1)
    label_data = json.load(f2)

merged_data = no_label_data + label_data

merged_data

[[{'subject': 'rivolta del pane',
   'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
   'object': 'https://www.wikidata.org/wiki/Q5465517'},
  {'subject': 'Maghreb',
   'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
   'object': 'https://www.wikidata.org/wiki/Q28227'},
  {'subject': 'Tunisia',
   'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
   'object': 'https://www.wikidata.org/wiki/Q948'},
  {'subject': 'Marocco',
   'predicate': 'http://www.w3.org/2002/07/owl/sameAs',
   'object': 'https://www.wikidata.org/wiki/Q1028'},
  {'subject': 'rivolta del pane',
   'predicate': 'http://sw-portal.deri.org/ontologies/swportal#hasLocation',
   'object': 'Maghreb'},
  {'subject': 'Tunisia',
   'predicate': 'http://purl.org/dc/elements/1.1/relation',
   'object': 'Marocco'},
  {'subject': 'Marocco',
   'predicate': 'http://purl.org/dc/elements/1.1/relation',
   'object': 'Tunisia'},
  {'subject': '50 morti in Tunisia',
   'predicate': 'http://www.europeana.eu/schemas/edm/countr

In [ ]:
print(type(merged_data[0]))

<class 'list'>


In [ ]:
def check_all_lists(merged_data):
    # Separate non-list items and list items
    graph_data = []
    filtered_lists = []

    for item in merged_data:
        if isinstance(item, list):
            filtered_lists.append(item)
        else:
            graph_data.append(item)

    # Flatten the list of lists
    flattened_lists = [element for sublist in filtered_lists for element in sublist]

    # Combine non-list items with flattened lists
    graph_data.extend(flattened_lists)

    res_list = [i for n, i in enumerate(graph_data)
            if i not in graph_data[:n]]

    return res_list

# Example usage
graph_data = check_all_lists(merged_data)
graph_data

[{'subject': 'Tunisia',
  'predicate': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'object': 'LOC'},
 {'subject': 'Tunisia',
  'predicate': 'http://purl.org/ontology/bibo/citedBy',
  'object': 1},
 {'subject': 'Algeria',
  'predicate': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'object': 'LOC'},
 {'subject': 'Algeria',
  'predicate': 'http://purl.org/ontology/bibo/citedBy',
  'object': 1},
 {'subject': 'Marocco',
  'predicate': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'object': 'LOC'},
 {'subject': 'Marocco',
  'predicate': 'http://purl.org/ontology/bibo/citedBy',
  'object': 1},
 {'subject': 'Magreb',
  'predicate': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'object': 'LOC'},
 {'subject': 'Magreb',
  'predicate': 'http://purl.org/ontology/bibo/citedBy',
  'object': 1},
 {'subject': 'Francia',
  'predicate': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'object': 'LOC'},
 {'subject': 'Francia',
  'predicate': 'http://purl.org/ontology/bib

In [ ]:
with open(merged_json, 'w') as f:
    json.dump(graph_data, f, indent=4)

# Visualize the Example Graph > create Edges and Nodes Dataframes from the articles with Id 1 and Id 5

In [ ]:
import pandas as pd

# Step 1: Extract unique nodes and create an ID for each
nodes = []
for triple in graph_data:
    nodes.append(triple['subject'])
    nodes.append(triple['object'])

# Remove duplicates and create a mapping from node label to a unique ID
unique_nodes = list(set(nodes))
node_id_map = {label: idx for idx, label in enumerate(unique_nodes)}

# Create a DataFrame for nodes with 'Id' and 'Label'
nodes_df = pd.DataFrame({
    'Id': list(node_id_map.values()),
    'Label': list(node_id_map.keys())
})

# Step 2: Create edges DataFrame using node IDs for 'source' and 'target'
edges = []
for triple in graph_data:
    edges.append({
        'source': node_id_map[triple['subject']],  # Use the ID for the source node
        'target': node_id_map[triple['object']],   # Use the ID for the target node
        'label': triple['predicate']               # Keep the predicate as the edge label
    })

edges_df = pd.DataFrame(edges)

In [ ]:
nodes_df

,Id,Label
0,0,Amartya Sen
1,1,Marocco
2,2,1
3,3,Norman Borlaug
4,4,https://www.wikidata.org/wiki/Q83823
...,...,...
60,60,Italia
61,61,https://www.wikidata.org/wiki/Q3982767
62,62,rivoluzione verde
63,63,https://www.wikidata.org/wiki/Q28227


In [ ]:
edges_df

,source,target,label
0,44,15,http://www.w3.org/1999/02/22-rdf-syntax-ns#type
1,44,2,http://purl.org/ontology/bibo/citedBy
2,17,15,http://www.w3.org/1999/02/22-rdf-syntax-ns#type
3,17,2,http://purl.org/ontology/bibo/citedBy
4,1,15,http://www.w3.org/1999/02/22-rdf-syntax-ns#type
...,...,...,...
81,50,64,http://vocab.getty.edu/ontology#ulan2779_owned_by
82,51,56,http://www.w3.org/2002/07/owl/sameAs
83,51,10,http://dbpedia.org/ontology/subsidiary
84,10,51,http://vocab.getty.edu/ontology#ulan2779_owned_by


In [ ]:
nodes_file_path = '/content/drive/MyDrive/Colab Notebooks/TESI/github/example_nodes_file.csv'
edges_file_path = '/content/drive/MyDrive/Colab Notebooks/TESI/github/example_edges_file.csv'

nodes_df.to_csv(nodes_file_path, index=False)
edges_df.to_csv(edges_file_path, index=False)

In [ ]:
# these dataframes are used to visualize the file in gephi